In [1]:
from collections import deque

# Define the variables (States and Territories)
states = ["WA", "NT", "SA", "Q", "NSW", "V", "T"]

# Define the domains (Possible colors)
domains = {state: ["Red", "Green", "Blue"] for state in states}

# Define the constraints (Adjacency rules)
adjacency = {
    "WA": ["NT", "SA"],
    "NT": ["WA", "SA", "Q"],
    "SA": ["WA", "NT", "Q", "NSW", "V"],
    "Q": ["NT", "SA", "NSW"],
    "NSW": ["Q", "SA", "V"],
    "V": ["SA", "NSW"],
    "T": []  # Tasmania has no neighbors
}

# Function to check if assignment is valid
def is_valid(state, color, assignment):
    for neighbor in adjacency[state]:
        if neighbor in assignment and assignment[neighbor] == color:
            return False
    return True

# Backtracking algorithm
def backtrack(assignment):
    if len(assignment) == len(states):
        return assignment
    
    unassigned = [s for s in states if s not in assignment]
    state = unassigned[0]  # Select the next state to color
    
    for color in domains[state]:
        if is_valid(state, color, assignment):
            assignment[state] = color
            result = backtrack(assignment)
            if result:
                return result
            assignment.pop(state)  # Backtrack if no solution found
    
    return None

# Solve the CSP
solution = backtrack({})
print("Map Coloring Solution:", solution)


Map Coloring Solution: {'WA': 'Red', 'NT': 'Green', 'SA': 'Blue', 'Q': 'Red', 'NSW': 'Green', 'V': 'Red', 'T': 'Red'}


In [5]:
from collections import defaultdict

states = ['WA', 'NT', 'Q', 'NSW', 'V', 'SA', 'T']
original_domains = {state: ['red', 'green', 'blue'] for state in states}
neighbors = {
    'WA': ['NT', 'SA'],
    'NT': ['WA', 'Q', 'SA'],
    'Q': ['NT', 'NSW', 'SA'],
    'NSW': ['Q', 'V', 'SA'],
    'V': ['NSW', 'SA'],
    'SA': ['WA', 'NT', 'Q', 'NSW', 'V'],
    'T': []
}

# Use a fresh copy for domains each time
domains = {state: list(colors) for state, colors in original_domains.items()}

def is_consistent(state, color, assignment):
    for neighbor in neighbors[state]:
        if neighbor in assignment and assignment[neighbor] == color:
            return False
    return True

def mrv(assignment):
    unassigned = [v for v in states if v not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))

def degree(var):
    return len([n for n in neighbors[var] if n not in domains])

def lcv(var):
    return sorted(domains[var], key=lambda val: sum(
        1 for neighbor in neighbors[var] if val in domains[neighbor]
    ))

def forward_check(assignment, var, value):
    inferences = []
    for neighbor in neighbors[var]:
        if neighbor not in assignment and value in domains[neighbor]:
            domains[neighbor].remove(value)
            inferences.append((neighbor, value))
            if not domains[neighbor]:
                # If any neighbor runs out of legal values, stop and backtrack
                for (n, val) in inferences:
                    domains[n].append(val)
                return False, []
    return True, inferences

def backtrack(assignment):
    if len(assignment) == len(states):
        return assignment
    var = mrv(assignment)
    for value in lcv(var):
        if is_consistent(var, value, assignment):
            assignment[var] = value
            success, inferences = forward_check(assignment, var, value)
            if success:
                result = backtrack(assignment)
                if result:
                    return result
            del assignment[var]
            for (neighbor, val) in inferences:
                domains[neighbor].append(val)
    return None

solution = backtrack({})
print("Solution:", solution)


Solution: {'WA': 'red', 'NT': 'green', 'SA': 'blue', 'Q': 'red', 'NSW': 'green', 'V': 'red', 'T': 'red'}
